In [1]:
# GPU 사용가능 여부 확인
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-09-13 16:11:24.786017: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2022-09-13 16:11:24.809072: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3600000000 Hz
2022-09-13 16:11:24.809417: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55f92f389e90 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-09-13 16:11:24.809429: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-09-13 16:11:24.810029: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-09-13 16:11:25.106648: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-13

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9668152483900564489,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12946716436188031915
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10801971200
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8798762900282956506
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5",
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 10817847296
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12810070603757266323
 physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:02:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14879283660600045457
 physical_device_desc: "device: XLA_GP

In [2]:
#파이썬 버전 확인
import sys
print(sys.version)

3.7.11 (default, Jul 27 2021, 14:32:16) 
[GCC 7.5.0]


In [3]:
#텐서플로우 버전 확인
import tensorflow as tf
tf.__version__

'1.15.0'

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "/device:GPU:1"

In [5]:
# common library
import pandas as pd
import numpy as np
import time
from stable_baselines.common.vec_env import DummyVecEnv
# config
from config.config_dji import *
# preprocessor
from preprocessing.preprocessors_dji import * # from folder.file import *
# model
from model.models_dji import *

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/home/kongminseok/anaconda3/envs/k/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [6]:
preprocessed_path = pd.read_csv("data/done_data.csv")
display(preprocessed_path.datadate.values)
display(len(preprocessed_path))

array([20090102, 20090102, 20090102, ..., 20200817, 20200817, 20200817])

87780

In [7]:
def run_model() -> None:
    """Train the model."""

    # read and preprocess data

    preprocessed_path = "data/done_data.csv"
    if os.path.exists(preprocessed_path):
        data = pd.read_csv(preprocessed_path, index_col=0) # 0번째 column을 인덱스로 지정
    else:
        data = preprocess_data()
        data = add_turbulence(data)
        data.to_csv(preprocessed_path)


    # print(data.head())
    # print(data.size)
    

    # 2015/10/01 is the date that validation starts 
    # 2016/01/01 is the date that real trading starts
    # unique_trade_date needs to start from 2015/10/01 for validation purpose
    # 하지만 코드에 따르면 2015/10/02부터 validation 시작
    
    # trade는 2020년 07월 6일까지 됨
    unique_trade_date = data[(data.datadate > 20151001)&(data.datadate <= 20200707)].datadate.unique()
    #print(unique_trade_date)

    # rebalance_window is the number of months to retrain the model
    # validation_window is the number of months to validation the model and select for trading
    rebalance_window = 63 # default=63
    validation_window = 63 # default=63
    
    
    ## Original Ensemble Strategy
    #run_ensemble_strategy(df=data, 
    #                      unique_trade_date= unique_trade_date,
    #                      rebalance_window = rebalance_window,
    #                      validation_window= validation_window)
    
    ## Remake Ensemble
    run_remake_ensemble(df=data, 
                  unique_trade_date= unique_trade_date,
                  rebalance_window = rebalance_window,
                  validation_window= validation_window)
    
    ## Renewal Ensemble2
    #run_ensemble2(df=data, 
    #              unique_trade_date= unique_trade_date,
    #              rebalance_window = rebalance_window,
    #              validation_window= validation_window)
    

    ## PPO
    #run_ppo(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    # A2C
    #run_a2c(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## DDPG
    #run_ddpg(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## ACKTR
    #run_acktr(df=data, 
    #       unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #       validation_window= validation_window)
    
    ## TRPO
    #run_trpo(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## SAC
    #run_sac(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## TD3
    #run_td3(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## GAIL
    #run_gail(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)


#     #_logger.info(f"saving model version: {_version}")
if __name__ == "__main__":
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        print(result)
        run_model()

result_dji_remake_HMAX_NORMALIZE10_hold0to3_3
============Start Remake Ensemble============
turbulence_threshold:  171.0940715631016
======Model training from:  20090000 to  20151002
======A2C Training========




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.




2022-09-13 16:11:25.809459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-13 16:11:25.809787: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:01:00.0
2022-09-13 16:11:25.809831: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-13 16:11:25.810109: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:02:00.0
2022-09-13 16:11:25.810136: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic li




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




2022-09-13 16:11:26.556416: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0


Training time (A2C):  3.4218868136405947  minutes
======A2C Validation from:  20151002 to  20160104
A2C Sharpe Ratio:  -0.11384743428794691
======PPO Training========



...

2022-09-14 03:00:23.499762: I tensorflow/core/profiler/lib/profiler_session.cc:205] Profiler session started.
2022-09-14 03:00:23.503089: I tensorflow/core/platform/default/device_tracer.cc:588] Collecting 119 kernel records, 35 memcpy records.
2022-09-14 03:00:25.357832: I tensorflow/core/profiler/lib/profiler_session.cc:205] Profiler session started.
2022-09-14 03:00:25.361739: I tensorflow/core/platform/default/device_tracer.cc:588] Collecting 119 kernel records, 35 memcpy records.
2022-09-14 03:00:27.314077: I tensorflow/core/profiler/lib/profiler_session.cc:205] Profiler session started.
2022-09-14 03:00:27.317647: I tensorflow/core/platform/default/device_tracer.cc:588] Collecting 119 kernel records, 35 memcpy records.
2022-09-14 03:00:29.187766: I tensorflow/core/profiler/lib/profiler_session.cc:205] Profiler session started.
2022-09-14 03:00:29.190894: I tensorflow/core/platform/default/device_tracer.cc:588] Collecting 119 kernel records, 35 memcpy records.
2022-09-14 03:00:31.

2022-09-14 03:01:28.545023: I tensorflow/core/profiler/lib/profiler_session.cc:205] Profiler session started.
2022-09-14 03:01:28.549165: I tensorflow/core/platform/default/device_tracer.cc:588] Collecting 119 kernel records, 35 memcpy records.


Training time (TRPO):  1.5945135513941446  minutes
======TRPO Validation from:  20200106 to  20200406
TRPO Sharpe Ratio:  -0.4938628005316281
======ACKTR Training========


2022-09-14 03:01:28.833807: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-14 03:01:28.834024: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:01:00.0
2022-09-14 03:01:28.834078: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-14 03:01:28.834388: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:02:00.0
2022-09-14 03:01:28.834413: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic li

Training time (ACKTR):  2.1493956089019775  minutes
======ACKTR Validation from:  20200106 to  20200406
ACKTR Sharpe Ratio:  -0.4382052456177623
======Trading from:  20200406 to  20200707
previous_total_asset:1259433.0160347805
end_total_asset:1259871.7021447804
total_reward:438.68610999989323
total cost:  45.49684
total trades:  86
total sells:  38
total buys:  48
Sharpe:  0.2909961002681229
Remake Ensemble took:  652.2046765128772  minutes


-----------